In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rf').getOrCreate()

In [3]:
data = spark.read.csv('College.csv',inferSchema=True,header = True)

In [4]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [8]:
output = assembler.transform(data)

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol='Private',outputCol='Privateindex')

In [11]:
output_fixed =  indexer.fit(output).transform(output)

In [12]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- Privateindex: double (nullable = false)



In [13]:
final_data = output_fixed.select('features','Privateindex')

In [14]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [ ]:
#from pyspark.ml.regression import RandomForestRegressor

In [16]:
from pyspark.ml import Pipeline

In [17]:
dtc = DecisionTreeClassifier(labelCol='Privateindex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='Privateindex',featuresCol='features')
gbt = GBTClassifier(labelCol='Privateindex',featuresCol='features')

In [18]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [20]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='Privateindex')

In [23]:
print('DTC')
print(my_binary_eval.evaluate(rfc_preds))

DTC
0.974926686217


In [24]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='Privateindex',rawPredictionCol='prediction')

In [25]:
print('gbt')
print(my_binary_eval2.evaluate(gbt_preds))

gbt
0.901173020528


In [26]:
from pyspark.ml.evaluation import  MulticlassClassificationEvaluator

In [28]:
acc_val = MulticlassClassificationEvaluator(labelCol='Privateindex',metricName='accuracy')

In [29]:
rfc_acc = acc_val.evaluate(rfc_preds)

In [30]:
rfc_acc

0.9118942731277533